In [17]:
import numpy as np
from sklearn.datasets import make_classification, load_iris, load_wine, load_breast_cancer
from sklearn.model_selection import train_test_split
import torch
from sklearn.model_selection import train_test_split



class ConformalPredictor:


    def __init__(self, model, alpha=0.05):
        self.model = model
        self.alpha = 0.05

    def fit(self, X, y, cal_size=0.33,**kwargs):
        X_train, X_cal, y_train, y_cal = train_test_split(X,y,test_size=cal_size)
        self.model.fit(X_train, y_train,**kwargs)
        y_pred_cal = self.model.predict_proba(X_cal)
        self.scores = 1 - y_pred_cal[np.arange(len(y_cal)), y_cal]
        n = len(self.scores)
        self.threshold = np.quantile(self.scores, np.ceil((n+1)*(1-self.alpha))/n, method="inverted_cdf")

    def predict_set(self,X):
        y_probas = self.model.predict_proba(X)
        pred_sets = []
        for y_proba in y_probas:
            pred_set = np.where(1 - y_proba <= self.threshold)[0]
            pred_sets.append(pred_set)
        return pred_sets
    

from simple_model import DyadOneHotPairDataset, DyadRankingModel, create_dyads

class ConformalRankingPredictor:
    def __init__(self, num_classes, alpha=0.05):
        self.num_classes = num_classes
        self.alpha = 0.05

    def fit(self, X, y, cal_size=0.33, hidden_dim=16, **kwargs):
        X_train, X_cal, y_train, y_cal = train_test_split(X,y,test_size=cal_size)
        self.model = DyadRankingModel(input_dim = X_train.shape[1] + y.max()+1, hidden_dim=16)
        self.model.fit(X_train, y_train,**kwargs)
        
        # # here we typically compute non conformity scores. For the ranker
        # # we use the predicted latent skill value
        
        # y_pred_cal = self.model.predict_proba(X_cal)
        # self.scores = 1 - y_pred_cal[np.arange(len(y_cal)), y_cal]
        cal_dyads = create_dyads(X_cal, y_cal, self.num_classes)
        with torch.no_grad():
            self.scores = self.model(cal_dyads).detach().numpy()
        n = len(self.scores)
        # TODO check alpha here
        self.threshold = np.quantile(self.scores, np.ceil((n+1)*(1-self.alpha))/n, method="inverted_cdf")

    def predict_set(self,X):
                
        y_skills = self.model.predict_class_skills(X)
        

        pred_sets = []
        for y_skill in y_skills:
            pred_set = np.where(1 - y_skill <= self.threshold)[0]
            pred_sets.append(pred_set)
            print(y_skill, self.threshold)
        return pred_sets

In [18]:
X, y = make_classification(n_samples=5000, n_informative = 4, n_classes=3, n_features=4, n_redundant=0, n_repeated=0)
# X,y = load_iris(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [19]:
from simple_model import ClassifierModel, DyadRankingModel
from sklearn.linear_model import LogisticRegression


clf = ClassifierModel(input_dim = X_train.shape[1], hidden_dim=16, output_dim=y.max()+1)


cp = ConformalPredictor(clf, alpha=0.1)

cp.fit(X_train, y_train, num_epochs=1000, learning_rate=0.01)

crp = ConformalRankingPredictor(num_classes=y_train.max()+1)
crp.fit(X_train,y_train, num_epochs=1000, learning_rate=0.01)

pred_sets_clf = cp.predict_set(X_test)
pred_sets_rnk = crp.predict_set(X_test)


[ -5.121089 -11.02523   -9.21367 ] 9.89567
[2.940431   8.74798    0.49976683] 9.89567
[-0.78709817 -3.6863697   2.9994624 ] 9.89567
[1.7094988  6.056782   0.42199516] 9.89567
[ -3.187609 -12.328981  -6.104479] 9.89567
[ 2.3940609  2.6586454 -6.9692993] 9.89567
[ 2.4337127  4.348509  -1.1853426] 9.89567
[ 4.7029324  3.7916572 -2.178284 ] 9.89567
[1.2436278 8.243591  1.5402052] 9.89567
[ 0.19280648  3.2415767  -3.5617006 ] 9.89567
[ 4.924412  -1.8027389 -4.429103 ] 9.89567
[-1.2752793 -6.746804  -7.9617043] 9.89567
[ 0.27053452 -2.695923    4.989361  ] 9.89567
[-3.1397288  7.3843126 -1.0138905] 9.89567
[ -0.8225095  -7.713456  -11.45904  ] 9.89567
[ 0.40764642 -2.44792     4.3910637 ] 9.89567
[-6.8701725 -8.676182  -3.3403184] 9.89567
[  1.7335756  -2.635022  -10.152511 ] 9.89567
[ -0.19623733  -4.868844   -12.987398  ] 9.89567
[ 0.8311746  1.1837707 -5.5002804] 9.89567
[ 1.1892276  1.2356551 -3.6478107] 9.89567
[8.18065   9.806243  3.2555554] 9.89567
[ 3.7675936 -2.3241713  5.9197817] 9

In [16]:

coverage_clf = np.mean([y_test[i] in pred_sets_clf[i] for i in range(len(y_test))])
efficiency_clf = np.mean([len(pred_sets_clf[i]) for i in range(len(y_test))])

coverage_rnk = np.mean([y_test[i] in pred_sets_rnk[i] for i in range(len(y_test))])
efficiency_rnk = np.mean([len(pred_sets_rnk[i]) for i in range(len(y_test))])

print(coverage_clf, efficiency_clf)
print(coverage_rnk, efficiency_rnk)

0.9515151515151515 1.0751515151515152
0.6618181818181819 1.5327272727272727
